In [1]:
import sys
import logging
import pandas as pd
import openai
import os
from dotenv import load_dotenv

sys.path.append('..')
import nature_go_client
from question_generation import generate_questions

logger = logging.getLogger(__name__)


load_dotenv()
NG_USERNAME = os.getenv("NG_USERNAME")
NG_PASSWORD = os.getenv("NG_PASSWORD")
openai.api_key = os.getenv("OPENAI_API_KEY")

In [2]:
client = nature_go_client.NatureGoClient(username=NG_USERNAME, password=NG_PASSWORD)
client.login()

## Retrieve species with missing questions through Nature go API

In [11]:
species_list = pd.DataFrame(client.get_labeled_species(limit=10, multiplechoicequestions=False))
species_list = species_list[species_list['num_observations_total'] > 0]
species_list

,id,scientificNameWithoutAuthor,commonNames,genus,family,gbif_id,powo_id,wikipedia_word_count,number_of_occurrences,occurences_cdf,...,descriptions,illustration,illustration_transparent,illustration_reference,illustration_reference_transparent,reference_image_url,illustration_url,display_name,num_observations_total,num_questions_total
0,1131,Hedera colchica,"[Persian Ivy, Colchis ivy, Caucasian Ivy]",Hedera,Araliaceae,3036033.0,90679-1,419.0,255.0,0.145775,...,"[Today, I discovered a fascinating plant calle...",http://nature-go.edouardleurent.com/media/spec...,http://nature-go.edouardleurent.com/media/spec...,None,None,,http://nature-go.edouardleurent.com/media/spec...,Persian Ivy,1,0
1,2271,Ocimum basilicum,"[Sweet basil, Basil, Thai Basil]",Ocimum,Lamiaceae,2927096.0,452874-1,2043.0,63.0,0.055122,...,[Basil is a fascinating herb that is used in c...,http://nature-go.edouardleurent.com/media/spec...,http://nature-go.edouardleurent.com/media/spec...,None,None,,http://nature-go.edouardleurent.com/media/spec...,Sweet basil,1,0
2,4127,Paeonia lactiflora,"[Common garden peony, Peony, Chinese peony]",Paeonia,Paeoniaceae,3083486.0,711802-1,490.0,14.0,0.019946,...,"[Today, I admired the beauty of Paeonia lactif...",None,None,None,None,,http://nature-go.edouardleurent.com/static/img...,Common garden peony,1,0
3,36470,Stellaria holostea,"[Greater stichwort, Greater Stitchwort, Easter...",Stellaria,Caryophyllaceae,,,NaN,NaN,NaN,...,"[Today, I saw the greater stitchwort, a beauti...",None,None,None,None,,http://nature-go.edouardleurent.com/static/img...,Greater stichwort,1,0


## Run generation with openai API

In [12]:
species = species_list.loc[0]
questions = generate_questions(species['display_name'])

In [13]:
questions

[{'question': 'What is the primary method of climbing for Persian Ivy?',
  'choices': ['Twining around objects',
   'Extending tendrils',
   'Using aerial rootlets',
   'Growing thorns'],
  'correct_choice': 2},
 {'question': 'Which type of leaves does Persian Ivy have on its creeping and climbing stems?',
  'choices': ['Palmately five-lobed juvenile leaves',
   'Unlobed cordate adult leaves',
   'Needle-like leaves',
   'Serrated leaves'],
  'correct_choice': 0},
 {'question': 'What is the primary food source for bees and other insects provided by Persian Ivy?',
  'choices': ['Flowers', 'Nectar', 'Fruits', 'Leaves'],
  'correct_choice': 1},
 {'question': 'In which type of microclimate does Persian Ivy prefer to grow?',
  'choices': ['Arid deserts',
   'Coastal areas',
   'Mountain ranges with cool shade',
   'Tropical rainforests'],
  'correct_choice': 2}]

## Upload result to Nature Go API

In [14]:
client.post_species_questions(int(species['id']), questions)

# Process the rest of the batch

In [16]:
species_list = pd.DataFrame(client.get_labeled_species(limit=10, multiplechoicequestions=False))
species_list = species_list[species_list['num_observations_total'] > 0]
for _, species in species_list.iterrows():
    questions = generate_questions(species['display_name'])
    client.post_species_questions(int(species['id']), questions)
    print(f"Uploaded results for {species['display_name']}")

API error occurred: This model's maximum context length is 4097 tokens. However, you requested 4168 tokens (3368 in the messages, 800 in the completion). Please reduce the length of the messages or completion. . Retrying in 1 second...
API error occurred: This model's maximum context length is 4097 tokens. However, you requested 4168 tokens (3368 in the messages, 800 in the completion). Please reduce the length of the messages or completion. . Retrying in 1 second...
Uploaded results for Sweet basil
API error occurred: This model's maximum context length is 4097 tokens. However, your messages resulted in 5318 tokens. Please reduce the length of the messages. . Retrying in 1 second...
API error occurred: This model's maximum context length is 4097 tokens. However, your messages resulted in 5186 tokens. Please reduce the length of the messages. . Retrying in 1 second...
Uploaded results for Common garden peony
Uploaded results for Greater stichwort
